#Objetivos


*   Extraer informacion de Jira
*   Procesar y crear Dataframes
*   Crear Dashboard estado actual proyectos
*   Crear un modelo predictivo para todas las variables del modelo







#Alcance

Abarca todos los proyectos creados en Jira dentro de una organizacion


#Premisas


1.   Comenzaremos trabajando solo con las finalizadas
2.   En segunda etapa se incorporaran el resto de las incidencias



#Estructura

El trabajo se ordena según los siguientes capitulos



#Obtención de Datos



##Using JIRA library for Python.



In [ ]:
#Instalamos la librería Jira
!pip install jira

###Carga de Librerias

In [ ]:
# import the installed Jira library
from jira import JIRA
from jira.resources import User
import re
from datetime import datetime
from dateutil import parser
from dateutil.parser import parse
import pytz
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA

###Validamos credenciales para acceder a Jira

In [ ]:
# Specify a server key. It should be your
# domain name link. yourdomainname.atlassian.net
jiraOptions = {'server': "https://XXX.atlassian.net"}

# Get a JIRA client instance, pass,
# Authentication parameters
# and the Server name.
# emailID = your emailID
# token = token you receive after registration
jira = JIRA(options=jiraOptions, basic_auth=("XXX@XXX.com", "XXX"))

**Estamos listos para analizar los datos!**

---



##Visión General

In [ ]:
# Visión holística de los Proyectos
projects = jira.projects()
# print(projects)
print("Cantidad de Proyectos:",len(projects)) # nro de proyectos en Jira

# Obtengo lista de proyectos
for proyecto in projects:
  print(proyecto.name)


Cantidad de Proyectos: 15
Campañas
Cierre contable
Copas
Demo del proyecto de asistencia
Errores a reportar
Gestión de asistencia al cliente
Matriz_Riesgo
My discovery project
Otro_intento_A
Proyecto Kanban
Proyecto_Registro_Tareas
Prueba-A
Soporte
Soporte incidencias TI
Stories


##Definir consulta JQL para los issues que nos interesan

Lo importante aquí es la consistencia en la consulta, es decir, filtrar issues/proyectos con la misma estructura

In [ ]:
# Definimos la consulta JQL para obtener los issues que nos interesan
jql = 'project = "OIA" and status="Listo"' # (uno en particular) #'projectType = "software"'#'project = "OIA"' #'project != null' (todos)#
issues = jira.search_issues(jql, maxResults=None)

print("Cantidad de Issues:",len(issues))
#print(issues)

Cantidad de Issues: 172


**Definimos las variables que nos interesan analizar**

##***Campos de las indicencias***

*   identificador-> issue.key
*   responsable->issue.fields.assignee.displayName
*   story_points->issue.fields.storypoints (!!)
*   prioridad-> ->issue.fields.priority
*   tipo_incidencia->issue.fields.issuetype
*   proyecto-> issue.fields.project
*   estado-> issue.fields.status
*   roles-> jira.project_roles()
*   tiempo_resolucion-> (issue.fields.resolutiondate - issue.fields.created).days , esto para service management, para soft creamos campo fecha y hora de resolucion
*   Presupuesto (costo) -> customfield_10055

Ver:
Bloqueando
Bloqueantes
HAY QUE HACER CONSULTA JQL issuelinktype in ('is blocked by') o Blocking


*A tener en cuenta:*

Para los campos personalizados, hay que mirar el html y buscar los customfield_XXXX

Describir el problema de story points: Revisando en foros, documentacion y html, no he logrado capturar el valor del campo story points, por lo que se puede utilizar otro campo que capture ese valor. En este caso, se creó el campo "horas estimadas de trabajo"

Pasamos a revisar entonces cada una de estas variables

###Identificador Incidencia

In [ ]:
incidencias_key=[]
for i in issues:
    key_issue=i.key
    incidencias_key.append(key_issue)

print(len(incidencias_key))
print(incidencias_key)

172
['OIA-369', 'OIA-368', 'OIA-367', 'OIA-366', 'OIA-365', 'OIA-364', 'OIA-363', 'OIA-362', 'OIA-361', 'OIA-360', 'OIA-359', 'OIA-358', 'OIA-353', 'OIA-352', 'OIA-351', 'OIA-350', 'OIA-349', 'OIA-348', 'OIA-347', 'OIA-346', 'OIA-333', 'OIA-332', 'OIA-331', 'OIA-325', 'OIA-324', 'OIA-323', 'OIA-322', 'OIA-321', 'OIA-320', 'OIA-319', 'OIA-318', 'OIA-317', 'OIA-316', 'OIA-315', 'OIA-314', 'OIA-313', 'OIA-312', 'OIA-311', 'OIA-310', 'OIA-309', 'OIA-308', 'OIA-307', 'OIA-305', 'OIA-304', 'OIA-303', 'OIA-302', 'OIA-301', 'OIA-300', 'OIA-299', 'OIA-298', 'OIA-297', 'OIA-296', 'OIA-295', 'OIA-294', 'OIA-293', 'OIA-292', 'OIA-291', 'OIA-290', 'OIA-289', 'OIA-288', 'OIA-287', 'OIA-286', 'OIA-285', 'OIA-284', 'OIA-283', 'OIA-282', 'OIA-281', 'OIA-280', 'OIA-279', 'OIA-278', 'OIA-277', 'OIA-268', 'OIA-267', 'OIA-266', 'OIA-254', 'OIA-236', 'OIA-235', 'OIA-234', 'OIA-233', 'OIA-232', 'OIA-231', 'OIA-230', 'OIA-229', 'OIA-228', 'OIA-227', 'OIA-226', 'OIA-225', 'OIA-224', 'OIA-223', 'OIA-222', 'OIA-

###Responsables

In [ ]:
#Obtén el usuario asignado para cada uno
asignados=[]
for issue in issues:
    # Verifica si el issue tiene un asignado
    if issue.fields.assignee is not None:
        # Accede al atributo displayName del objeto User del atributo assignee
        assignee_name = issue.fields.assignee.displayName
        asignados.append(assignee_name)
    else:
        assignee_name='Sin Asignar'
        asignados.append(assignee_name)

print(len(asignados))
print(asignados)

172
['Valentin Correa', 'Sin Asignar', 'Valentin Correa', 'Sin Asignar', 'Valentin Correa', 'Sin Asignar', 'Valentin Correa', 'Valentin Correa', 'Valentin Correa', 'Sin Asignar', 'Valentin Correa', 'Sin Asignar', 'Valentin Correa', 'Sin Asignar', 'Valentin Correa', 'Sin Asignar', 'Valentin Correa', 'Sin Asignar', 'Valentin Correa', 'Sin Asignar', 'Valentin Correa', 'Sin Asignar', 'Valentin Correa', 'Valentin Correa', 'Sin Asignar', 'Valentin Correa', 'Sin Asignar', 'Valentin Correa', 'Sin Asignar', 'Valentin Correa', 'Valentin Correa', 'Valentin Correa', 'Sin Asignar', 'Valentin Correa', 'Sin Asignar', 'Valentin Correa', 'Sin Asignar', 'Valentin Correa', 'Sin Asignar', 'Valentin Correa', 'Sin Asignar', 'Valentin Correa', 'Valentin Correa', 'Sin Asignar', 'Valentin Correa', 'Sin Asignar', 'Valentin Correa', 'Sin Asignar', 'Valentin Correa', 'Sin Asignar', 'Valentin Correa', 'Sin Asignar', 'Valentin Correa', 'Sin Asignar', 'Valentin Correa', 'Valentin Correa', 'Valentin Correa', 'Sin Asi

###Story Points

In [ ]:
points=[] # customfield_10051 es "Horas estimadas de trabajo", replica el valor de story points con una automatizacion
for i in issues:

    if i.fields.customfield_10051 is not None:
        punto = i.fields.customfield_10051
        points.append(punto)
    else:
        punto=0
        points.append(punto)

print(len(points))
print(points)


172
[0, 0, 0, 0, 0, 0, 2.0, 5.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 9.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 20.0, 5.0, 0, 25.0, 0, 0, 0, 0, 0, 0, 0, 10.0, 0, 0, 0, 0, 0, 7.0, 2.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3.0, 2.0, 4.0, 9.0, 1.0, 10.0, 6.0, 17.0, 8.0, 0, 20.0, 28.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


###Prioridad

In [ ]:
incidencias_prioridad=[]
for i in issues:
    prioridad=i.fields.priority.name
    incidencias_prioridad.append(prioridad)

print(len(incidencias_prioridad))
print(incidencias_prioridad)

172
['Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medi

###Tipo de Incidencia

In [ ]:
incidencias_tipo=[]
for i in issues:
    tipo=i.fields.issuetype.name
    incidencias_tipo.append(tipo)

print(len(incidencias_tipo))
print(incidencias_tipo)

172
['Subtarea', 'Subtarea', 'Subtarea', 'Actividad', 'Subtarea', 'Subtarea', 'Subtarea', 'Actividad', 'Subtarea', 'Subtarea', 'Subtarea', 'Actividad', 'Subtarea', 'Subtarea', 'Subtarea', 'Actividad', 'Subtarea', 'Subtarea', 'Subtarea', 'Actividad', 'Subtarea', 'Subtarea', 'Subtarea', 'Subtarea', 'Subtarea', 'Subtarea', 'Actividad', 'Subtarea', 'Subtarea', 'Subtarea', 'Actividad', 'Subtarea', 'Subtarea', 'Subtarea', 'Actividad', 'Subtarea', 'Subtarea', 'Subtarea', 'Actividad', 'Subtarea', 'Subtarea', 'Subtarea', 'Subtarea', 'Subtarea', 'Subtarea', 'Actividad', 'Subtarea', 'Subtarea', 'Subtarea', 'Actividad', 'Subtarea', 'Subtarea', 'Subtarea', 'Actividad', 'Subtarea', 'Subtarea', 'Subtarea', 'Actividad', 'Subtarea', 'Subtarea', 'Subtarea', 'Actividad', 'Subtarea', 'Subtarea', 'Subtarea', 'Actividad', 'Subtarea', 'Subtarea', 'Subtarea', 'Subtarea', 'Actividad', 'Subtarea', 'Subtarea', 'Subtarea', 'Subtarea', 'Subtarea', 'Subtarea', 'Subtarea', 'Actividad', 'Subtarea', 'Subtarea', 'Subta

###Proyecto

In [ ]:
incidencias_proyecto=[]
for i in issues:
    proyecto=i.fields.project.name
    incidencias_proyecto.append(proyecto)

print(len(incidencias_proyecto))
print(incidencias_proyecto)

172
['Otro_intento_A', 'Otro_intento_A', 'Otro_intento_A', 'Otro_intento_A', 'Otro_intento_A', 'Otro_intento_A', 'Otro_intento_A', 'Otro_intento_A', 'Otro_intento_A', 'Otro_intento_A', 'Otro_intento_A', 'Otro_intento_A', 'Otro_intento_A', 'Otro_intento_A', 'Otro_intento_A', 'Otro_intento_A', 'Otro_intento_A', 'Otro_intento_A', 'Otro_intento_A', 'Otro_intento_A', 'Otro_intento_A', 'Otro_intento_A', 'Otro_intento_A', 'Otro_intento_A', 'Otro_intento_A', 'Otro_intento_A', 'Otro_intento_A', 'Otro_intento_A', 'Otro_intento_A', 'Otro_intento_A', 'Otro_intento_A', 'Otro_intento_A', 'Otro_intento_A', 'Otro_intento_A', 'Otro_intento_A', 'Otro_intento_A', 'Otro_intento_A', 'Otro_intento_A', 'Otro_intento_A', 'Otro_intento_A', 'Otro_intento_A', 'Otro_intento_A', 'Otro_intento_A', 'Otro_intento_A', 'Otro_intento_A', 'Otro_intento_A', 'Otro_intento_A', 'Otro_intento_A', 'Otro_intento_A', 'Otro_intento_A', 'Otro_intento_A', 'Otro_intento_A', 'Otro_intento_A', 'Otro_intento_A', 'Otro_intento_A', 'Otro

###Estado de las Incidencias

In [ ]:
incidencias_estado=[]
for i in issues:
    estado=i.fields.status.name
    incidencias_estado.append(estado)

print(len(incidencias_estado))
print(incidencias_estado)

172
['Listo', 'Listo', 'Listo', 'Listo', 'Listo', 'Listo', 'Listo', 'Listo', 'Listo', 'Listo', 'Listo', 'Listo', 'Listo', 'Listo', 'Listo', 'Listo', 'Listo', 'Listo', 'Listo', 'Listo', 'Listo', 'Listo', 'Listo', 'Listo', 'Listo', 'Listo', 'Listo', 'Listo', 'Listo', 'Listo', 'Listo', 'Listo', 'Listo', 'Listo', 'Listo', 'Listo', 'Listo', 'Listo', 'Listo', 'Listo', 'Listo', 'Listo', 'Listo', 'Listo', 'Listo', 'Listo', 'Listo', 'Listo', 'Listo', 'Listo', 'Listo', 'Listo', 'Listo', 'Listo', 'Listo', 'Listo', 'Listo', 'Listo', 'Listo', 'Listo', 'Listo', 'Listo', 'Listo', 'Listo', 'Listo', 'Listo', 'Listo', 'Listo', 'Listo', 'Listo', 'Listo', 'Listo', 'Listo', 'Listo', 'Listo', 'Listo', 'Listo', 'Listo', 'Listo', 'Listo', 'Listo', 'Listo', 'Listo', 'Listo', 'Listo', 'Listo', 'Listo', 'Listo', 'Listo', 'Listo', 'Listo', 'Listo', 'Listo', 'Listo', 'Listo', 'Listo', 'Listo', 'Listo', 'Listo', 'Listo', 'Listo', 'Listo', 'Listo', 'Listo', 'Listo', 'Listo', 'Listo', 'Listo', 'Listo', 'Listo', 'List

###Presupuesto

In [ ]:
presupuesto=[]
for i in issues:
    valor=i.fields.customfield_10055
    presupuesto.append(valor)

print(len(presupuesto))
print(presupuesto)

172
[20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 80.0, 80.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 120.0, 20.0, 20.0, 120.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 400.0, 100.0, 20.0, 500.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 0.0, 140.0, 40.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 20.0, 

###Tiempo de Creación, Cierre y Tiempo de Trabajo

In [ ]:
#Fecha y hora de Creacion
incidencias_tiempo_resol=[]
for i in issues:
    fecha_hora_jira=i.fields.created
    # Parsear la cadena de texto a un objeto datetime
    dt = parser.parse(fecha_hora_jira)
    # Obtener la fecha, hora y minutos
    fecha_hora_minutos = dt.strftime('%d-%m-%Y %H:%M') #%d-%m-%Y%H:%M
    incidencias_tiempo_resol.append(fecha_hora_minutos)

print(len(incidencias_tiempo_resol))
print(incidencias_tiempo_resol)

172
['28-09-2023 15:49', '28-09-2023 15:49', '28-09-2023 15:48', '28-09-2023 15:48', '30-08-2023 09:43', '30-08-2023 09:43', '30-08-2023 09:43', '30-08-2023 09:43', '14-06-2023 13:13', '14-06-2023 13:13', '14-06-2023 13:13', '14-06-2023 13:13', '05-05-2023 15:40', '05-05-2023 15:40', '05-05-2023 15:40', '05-05-2023 15:40', '29-04-2023 11:32', '29-04-2023 11:32', '29-04-2023 11:32', '29-04-2023 11:32', '29-04-2023 09:36', '29-04-2023 09:36', '29-04-2023 09:36', '28-04-2023 20:27', '28-04-2023 20:27', '28-04-2023 20:27', '28-04-2023 20:27', '18-04-2023 14:52', '18-04-2023 14:52', '18-04-2023 14:52', '18-04-2023 14:52', '18-04-2023 14:37', '18-04-2023 14:37', '18-04-2023 14:37', '18-04-2023 14:37', '14-04-2023 21:07', '14-04-2023 21:07', '14-04-2023 21:07', '14-04-2023 21:07', '12-04-2023 19:47', '12-04-2023 19:47', '12-04-2023 19:47', '13-03-2023 20:20', '13-03-2023 20:20', '13-03-2023 20:20', '13-03-2023 20:20', '28-11-2022 13:29', '28-11-2022 13:29', '28-11-2022 13:29', '28-11-2022 13:

In [ ]:
#Fecha y hora de Cierre   -> Fecha y Hora de resolucion (customfield_10077)
incidencias_tiempo_cierre=[]
for i in issues:
    if i.fields.customfield_10077 is not None:
      momento_cierre=i.fields.customfield_10077
      # Parsear la cadena de texto a un objeto datetime
      dt_2 = parser.parse(momento_cierre)
      # Obtener la fecha, hora y minutos
      fecha_hora_minutos = dt_2.strftime('%d-%m-%Y %H:%M') #%d-%m-%Y%H:%M
      incidencias_tiempo_cierre.append(fecha_hora_minutos)
    else:
      momento_cierre="0"
      incidencias_tiempo_cierre.append(momento_cierre)

print(len(incidencias_tiempo_cierre))
print(incidencias_tiempo_cierre)

# Aquellas incidencias con valor 0 indican que estan abiertas
# En este caso trabajamos solamente con incidencias cerradas

172
['28-09-2023 15:49', '28-09-2023 15:49', '28-09-2023 15:50', '28-09-2023 15:50', '30-08-2023 09:44', '30-08-2023 09:44', '30-08-2023 09:44', '30-08-2023 09:44', '14-06-2023 13:14', '14-06-2023 13:14', '14-06-2023 13:14', '14-06-2023 13:14', '08-05-2023 16:06', '08-05-2023 16:11', '05-05-2023 15:54', '08-05-2023 16:11', '05-05-2023 15:54', '08-05-2023 16:18', '08-05-2023 16:17', '08-05-2023 16:18', '29-04-2023 09:38', '29-04-2023 09:38', '29-04-2023 09:38', '28-04-2023 20:29', '28-04-2023 20:29', '28-04-2023 20:29', '28-04-2023 20:29', '20-04-2023 19:31', '20-04-2023 19:31', '20-04-2023 19:31', '20-04-2023 19:31', '20-04-2023 19:31', '20-04-2023 19:31', '20-04-2023 19:31', '20-04-2023 19:31', '20-04-2023 19:31', '20-04-2023 19:31', '20-04-2023 19:31', '20-04-2023 19:31', '20-04-2023 19:31', '20-04-2023 19:31', '20-04-2023 19:31', '20-04-2023 19:31', '20-04-2023 19:31', '20-04-2023 19:31', '20-04-2023 19:31', '20-04-2023 19:31', '20-04-2023 19:31', '20-04-2023 19:31', '20-04-2023 19:

In [ ]:
#Calculamos los dias laborables
dias_lab=[]

for issue in issues:
    # Obtén las fechas de inicio y finalización directamente desde el objeto 'issue'
    fecha_inicio = pd.to_datetime(issue.fields.created)
    fecha_fin = pd.to_datetime(issue.fields.customfield_10077)

    # Genera un rango de fechas diarias entre las fechas de inicio y finalización
    rango_fechas = pd.date_range(start=fecha_inicio, end=fecha_fin)

    # Define una función para verificar si un día es laborable (lunes a viernes)
    def es_dia_laborable(fecha):
        return fecha.weekday() < 5  # 0 corresponde a lunes, 1 a martes, y así sucesivamente

    # Filtra las fechas para obtener solo los días laborables
    dias_laborables = [fecha for fecha in rango_fechas if es_dia_laborable(fecha)]

    # Calcula la cantidad de días laborables
    cantidad_dias_laborables = len(dias_laborables)
    dias_lab.append(cantidad_dias_laborables)

print(len(dias_lab))
print(dias_lab)

172
[0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 2, 5, 6, 6, 6, 0, 0, 0, 1, 1, 1, 1, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 6, 6, 6, 28, 28, 28, 28, 104, 104, 104, 104, 109, 109, 109, 109, 117, 117, 117, 117, 117, 117, 117, 117, 117, 117, 117, 117, 117, 117, 117, 117, 117, 120, 120, 120, 121, 136, 136, 136, 136, 136, 136, 136, 136, 136, 136, 136, 136, 136, 136, 136, 136, 136, 136, 136, 136, 138, 138, 138, 138, 148, 148, 148, 148, 148, 155, 155, 155, 155, 155, 155, 155, 155, 155, 155, 155, 155, 155, 155, 155, 155, 156, 156, 156, 156, 156, 156, 156, 156, 157, 157, 157, 157, 157, 157, 157, 157, 157, 157, 157, 158, 160, 161, 208, 208, 208, 208, 208, 208, 208, 208, 209, 209, 209, 209, 209, 209, 209, 209, 209, 209, 209, 209, 209, 209, 209, 209, 209, 210, 210, 210, 210, 210]


##Armado del Dataframe

In [ ]:
# Creamos DF
a={'Proyecto':incidencias_proyecto,'N°_incidencia':incidencias_key,'Responsable':asignados,'SP':points,'Prioridad':incidencias_prioridad,'Tipo_incidencia':incidencias_tipo,'Estado':incidencias_estado,'Costo':presupuesto,'Inicio':incidencias_tiempo_resol,'Cierre':incidencias_tiempo_cierre,'Dias_Laborales':dias_lab}
df=pd.DataFrame(a, index=list(range(1,len(incidencias_proyecto)+1)))

In [ ]:
df

,Proyecto,N°_incidencia,Responsable,SP,Prioridad,Tipo_incidencia,Estado,Costo,Inicio,Cierre,Dias_Laborales
1,Otro_intento_A,OIA-369,Valentin Correa,0.0,Medium,Subtarea,Listo,20.0,28-09-2023 15:49,28-09-2023 15:49,0
2,Otro_intento_A,OIA-368,Sin Asignar,0.0,Medium,Subtarea,Listo,20.0,28-09-2023 15:49,28-09-2023 15:49,0
3,Otro_intento_A,OIA-367,Valentin Correa,0.0,Medium,Subtarea,Listo,20.0,28-09-2023 15:48,28-09-2023 15:50,1
4,Otro_intento_A,OIA-366,Sin Asignar,0.0,Medium,Actividad,Listo,20.0,28-09-2023 15:48,28-09-2023 15:50,1
5,Otro_intento_A,OIA-365,Valentin Correa,0.0,Medium,Subtarea,Listo,20.0,30-08-2023 09:43,30-08-2023 09:44,1
...,...,...,...,...,...,...,...,...,...,...,...
168,Otro_intento_A,OIA-6,Sin Asignar,0.0,Medium,Actividad,Listo,20.0,01-07-2022 16:10,20-04-2023 19:31,210
169,Otro_intento_A,OIA-5,Valentin Correa,0.0,Medium,Actividad,Listo,20.0,01-07-2022 15:51,20-04-2023 19:31,210
170,Otro_intento_A,OIA-4,Sin Asignar,0.0,Medium,Actividad,Listo,20.0,01-07-2022 15:24,20-04-2023 19:31,210
171,Otro_intento_A,OIA-2,Sin Asignar,0.0,Medium,Actividad,Listo,20.0,01-07-2022 15:21,20-04-2023 19:31,210


In [ ]:
'''
# Guardamos DF en un csv o excel
nombre_file=f"df_jira.xlsx"

df.to_excel(nombre_file,index=False)

# PARA DESCARGAR EL ARCHIVO GENERADO
from google.colab import files
files.download(nombre_file)
'''

'\n# Guardamos DF en un csv o excel\nnombre_file=f"df_jira.xlsx"\n\ndf.to_excel(nombre_file,index=False)\n\n# PARA DESCARGAR EL ARCHIVO GENERADO\nfrom google.colab import files\nfiles.download(nombre_file)\n'

#Data Understanding / Exploratory Data Analysis (EDA)

##Medias de resumen (var num)

In [ ]:
#Dimension del DF (filas x col)
print(df.shape)

(172, 11)


In [ ]:
#Resumen estadístico
df.describe(include='all')

,Proyecto,N°_incidencia,Responsable,SP,Prioridad,Tipo_incidencia,Estado,Costo,Inicio,Cierre,Dias_Laborales
count,172,172,172,172.000000,172,172,172,172.000000,172,172,172.000000
unique,1,172,2,NaN,1,2,1,NaN,86,12,NaN
top,Otro_intento_A,OIA-369,Sin Asignar,NaN,Medium,Subtarea,Listo,NaN,13-10-2022 11:43,20-04-2023 19:31,NaN
freq,172,1,114,NaN,172,102,172,NaN,8,145,NaN
mean,NaN,NaN,NaN,1.156977,NaN,NaN,NaN,28.023256,NaN,NaN,114.819767
std,NaN,NaN,NaN,4.120099,NaN,NaN,NaN,49.116636,NaN,NaN,72.902379
min,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,0.000000,NaN,NaN,0.000000
25%,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,20.000000,NaN,NaN,28.000000
50%,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,20.000000,NaN,NaN,136.000000
75%,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,20.000000,NaN,NaN,157.000000


### Variables numéricas y categóricas

In [ ]:
# Obtener la cantidad de variables numéricas y categóricas
num_vars = len(df.select_dtypes(include=['float', 'int']).columns)
cat_vars = len(df.select_dtypes(include=['object', 'category']).columns)

# Imprimir la cantidad de variables numéricas y categóricas
print('Cantidad de variables numéricas:', num_vars)
print('Cantidad de variables categóricas:', cat_vars)

Cantidad de variables numéricas: 3
Cantidad de variables categóricas: 8


In [ ]:
# Lista de Variables y sus tipos
df.dtypes

Proyecto            object
N°_incidencia       object
Responsable         object
SP                 float64
Prioridad           object
Tipo_incidencia     object
Estado              object
Costo              float64
Inicio              object
Cierre              object
Dias_Laborales       int64
dtype: object

Vamos a transformar a datetime las variables de inicio y fin

In [ ]:
df['Inicio'] = pd.to_datetime(df['Inicio'],format='%d-%m-%Y %H:%M')

df['Cierre'] = pd.to_datetime(df['Cierre'], errors='coerce')

# Reemplazamos los valores 0 por NaT
df['Cierre'].replace(0, np.nan, inplace=True)

Validamos la correcta transformacion

In [ ]:
df.dtypes

Proyecto                   object
N°_incidencia              object
Responsable                object
SP                        float64
Prioridad                  object
Tipo_incidencia            object
Estado                     object
Costo                     float64
Inicio             datetime64[ns]
Cierre             datetime64[ns]
Dias_Laborales              int64
dtype: object

###Datos atípicos o outliers

In [ ]:
# Crear un nuevo dataframe utilizando las variables seleccionadas
df_nuevo = df[['SP','Dias_Laborales']].copy()

# Definir una función para calcular el porcentaje de valores atípicos
def porcentaje_atipicos(columna):
    Q1 = columna.quantile(0.25)
    Q3 = columna.quantile(0.75)
    IQR = Q3 - Q1
    atipicos = ((columna < (Q1 - 1.5 * IQR)) | (columna > (Q3 + 1.5 * IQR))).sum()
    porcentaje = (atipicos / len(columna)) * 100
    return porcentaje

# Calcular el porcentaje de valores atípicos para cada variable
for columna in df_nuevo.columns:
    porcentaje = porcentaje_atipicos(df_nuevo[columna])
    print(f"Variable {columna}: {porcentaje}% de valores atípicos")


Variable SP: 12.209302325581394% de valores atípicos
Variable Dias_Laborales: 0.0% de valores atípicos


Podemos considerar tolerable el % de outliers (??)

###Datos faltantes / Valores perdidos

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 172 entries, 1 to 172
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Proyecto         172 non-null    object        
 1   N°_incidencia    172 non-null    object        
 2   Responsable      172 non-null    object        
 3   SP               172 non-null    float64       
 4   Prioridad        172 non-null    object        
 5   Tipo_incidencia  172 non-null    object        
 6   Estado           172 non-null    object        
 7   Costo            172 non-null    float64       
 8   Inicio           172 non-null    datetime64[ns]
 9   Cierre           172 non-null    datetime64[ns]
 10  Dias_Laborales   172 non-null    int64         
dtypes: datetime64[ns](2), float64(2), int64(1), object(6)
memory usage: 16.1+ KB


In [ ]:
# Verificar valores nulos en el DataFrame
valores_nulos = df.isnull()

# Contar valores nulos por columna
valores_nulos_por_columna = valores_nulos.sum()

# Contar valores nulos en total
total_valores_nulos = valores_nulos.sum().sum()

# Imprimir los resultados
print("Valores nulos por columna:")
print(valores_nulos_por_columna)
print("\nTotal de valores nulos:", total_valores_nulos)


Valores nulos por columna:
Proyecto           0
N°_incidencia      0
Responsable        0
SP                 0
Prioridad          0
Tipo_incidencia    0
Estado             0
Costo              0
Inicio             0
Cierre             0
Dias_Laborales     0
dtype: int64

Total de valores nulos: 0


In [ ]:
missing = df.isnull()

# calcular correlación entre variables con valores no faltantes
corr_matrix = df[~missing].corr(method='pearson')

# mostrar matriz de correlación
print(corr_matrix)


                      SP     Costo  Dias_Laborales
SP              1.000000  0.586727       -0.000003
Costo           0.586727  1.000000       -0.054371
Dias_Laborales -0.000003 -0.054371        1.000000


<ipython-input-29-84e13a909d7c>:4: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  corr_matrix = df[~missing].corr(method='pearson')


No hay correlacion entre variables con valores no faltantes

#Data Preparation / Cleaning / Feature Engineering

Acá la clave es definir el problema a resolver!

Recordar incorporar la metricas tipicas de los proyectos:

En definitiva, se trata de
1. Monitor estado actual
- Total de proyectos
-Total de tareas
-Estado de las tareas
-User workload
-Issue by type
-Presupuesto
-Progreso


2. Modelos predictivos
- Identificar las tareas que tardan más tiempo en completarse.
-Identificar los cuellos de botella en los proyectos.
-Identificar los usuarios que tienen dificultades para completar las tareas.
-Identificar las tareas que son más propensas a errores.



##Monitor Estado Actual de Proyectos

### User workload

In [ ]:
workload = pd.crosstab(df['Proyecto'], df['Responsable'])

print(workload)

Responsable     Sin Asignar  Valentin Correa
Proyecto                                    
Otro_intento_A          114               58


### Issue by type

In [ ]:
issue__type = pd.crosstab(df['Proyecto'], df['Tipo_incidencia'])

print(issue__type)

Tipo_incidencia  Actividad  Subtarea
Proyecto                            
Otro_intento_A          70       102


### Budget

In [ ]:
# Agrupamos el DataFrame por proyecto.
df_agrupado = df.groupby("Proyecto")

# Calculamos el costo total de cada proyecto.
costo_total = df_agrupado["Costo"].sum()

# Creamos un nuevo DataFrame con el costo total de cada proyecto.
df_costo_total = pd.DataFrame({
        "Proyecto": costo_total.index,
        "Costo total": costo_total.values
    })

print(df_costo_total)

         Proyecto  Costo total
0  Otro_intento_A       4820.0


### Progress -- Issues Not Done

In [ ]:
def tareas_no_listas(estados):

  return len([estado for estado in estados if estado != "Listo"])


estados = df['Estado']

tareas_no_listas = tareas_no_listas(estados)

print(tareas_no_listas)

0
